# Sesión 2 Databricks - Introducción a Apache Spark: RDDs, DataFrames, Datasets y Spark SQL


Todo el contenido de esta sesión puede ser amplicado con: 
- https://docs.databricks.com/aws/en/
- https://spark.apache.org/docs/latest/api/python/index.html
- https://learn.microsoft.com/en-us/azure/databricks/notebooks/best-practices

## Tipos de  celdas

En Databricks, podemos crear celdas de lops siguientes tipos:
* Markdown
* Python
* R
* Scala
* SQL
* Bash

In [0]:
# %scala
# object HelloWorld {
#   def main(args: Array[String]): Unit = {
#     println("Hello, World!")
#   }
# }

In [0]:
%sh
ls -la /

total 128
drwxr-xr-x    1 root   root         4096 May 11 17:37 .
drwxr-xr-x    1 root   root         4096 May 11 17:37 ..
lrwxrwxrwx    1 root   root            7 Jan 11  2024 bin -> usr/bin
drwxr-xr-x    2 root   root         4096 Apr 18  2022 boot
-r-xr-xr-x    1 root   root          228 Jan  1  1970 BUILD.bazel
drwxr-xr-x+   1 root   spark-users  4096 May 11 17:43 databricks
lrwxrwxrwx    1 root   root           25 May 11 17:37 databricks-datasets -> /dbfs/databricks-datasets
drwxrwxrwx    2 nobody nogroup      4096 May 11 17:37 dbfs
drwxrwxrwx    2 root   root         4096 May 11 17:37 dbr_utils_shared_empty_dir
drwxr-xr-x    5 root   root          380 May 11 17:42 dev
drwxr-xr-x    1 root   root         4096 May 11 18:13 etc
drwxr-xr-x+   1 root   spark-users  4096 May 11 18:13 home
lrwxrwxrwx    1 root   root            7 Jan 11  2024 lib -> usr/lib
lrwxrwxrwx    1 root   root            9 Jan 11  2024 lib32 -> usr/lib32
lrwxrwxrwx    1 root   root            9 Jan 11  2024 lib6

In [0]:
# %r
# x <- sqrt(18)
# print(x)

In [0]:
print("Hola mundo!")

Hola mundo!


## dbutils

dbutils es una utilidad interna de Databricks que proporciona comandos para interactuar con el entorno de ejecución de notebooks. Es muy útil para tareas como manipular archivos en DBFS, trabajar con secretos, ejecutar notebooks dentro de otros notebooks y más.

In [0]:
dbutils.help()

This module provides various utilities for users to interact with the rest of Databricks.
 credentials: DatabricksCredentialUtils -> Utilities for interacting with credentials within notebooks data: DataUtils -> Utilities for understanding and interacting with datasets (EXPERIMENTAL) fs: DbfsUtils -> Manipulates the Databricks filesystem (DBFS) from the console jobs: JobsUtils -> Utilities for leveraging jobs features library: LibraryUtils -> Utilities for session isolated libraries meta: MetaUtils -> Methods to hook into the compiler (EXPERIMENTAL) notebook: NotebookUtils -> Utilities for the control flow of a notebook (EXPERIMENTAL) preview: Preview -> Utilities under preview category secrets: SecretUtils -> Provides utilities for leveraging secrets within notebooks widgets: WidgetsUtils -> Methods to create and get bound value of input widgets inside notebooks

### dbutils.secrets
Para usar secretos, por detrás se puede almacenar en 
* Azure Key Vault.
* Base de datos administrada por Databricks.
* AWS Secrets Manager (IAM Rol configurado para poder acceder).

In [0]:
dbutils.secrets.help()

Provides utilities for leveraging secrets within notebooks.
Databricks documentation for more info.
 get(scope: String, key: String): String -> Gets the string representation of a secret value with scope and key getBytes(scope: String, key: String): byte[] -> Gets the bytes representation of a secret value with scope and key list(scope: String): Seq -> Lists secret metadata for secrets within a scope listScopes: Seq -> Lists secret scopes

### dbutils.fs
Para interactuar con el sistema de archivos, utilizamos *dbutils.fs*, comandos de interés:
* dbutils.fs.cp
* dbutils.fs.ls
* dbutils.fs..mv
* dbutils.fs.rm

In [0]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point updateMount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Similar to mount(), but updates an existing mount point (if present) instead of creating a new one fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory

In [0]:
#dbutils.fs.mounts()

In [0]:
dbutils.fs.ls("/")

[FileInfo(path='dbfs:/Volumes/', name='Volumes/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/databricks-datasets/', name='databricks-datasets/', size=0, modificationTime=0)]

### dbutils.notebook
Para poder interactuar con otros notebooks, utilizamos *dbutils.notebook*

In [0]:
%python
dbutils.notebook.help()

The notebook module.
 exit(value: String): void -> This method lets you exit a notebook with a value run(path: String, timeoutSeconds: int, arguments: Map): String -> This method runs a notebook and returns its exit value

### dbutils.widgets
Para poder paramterizar los notebooks, se utilizan los widgets, que hay de 4 tipos:
* text
* dropdown
* combobox
* multiselect

In [0]:
dbutils.widgets.text("schema_name", "databricks_david_schema")
# dbutils.widgets.dropdown("state", "CA", ["CA", "IL", "MI", "NY", "OR", "VA"])
# dbutils.widgets.combobox(
#   name='fruits_combobox',
#   defaultValue='banana',
#   choices=['apple', 'banana', 'coconut', 'dragon fruit'],
#   label='Fruits'
# )
# dbutils.widgets.multiselect(
#   name='days_multiselect',
#   defaultValue='Tuesday',
#   choices=['Monday', 'Tuesday', 'Wednesday', 'Thursday',
#     'Friday', 'Saturday', 'Sunday'],
#   label='Days of the Week'
# )


Para obtener el valor del parámetro establecido:

In [0]:
dbutils.widgets.get("schema_name") 

'databricks_david_schema'

Para obtener como un diccionario en Python, todos los parámetros establecidos:


In [0]:
dbutils.widgets.getAll()

{'volume_name': 'demo_volume',
 'catalog_name': 'demo_catalog',
 'schema_name': 'databricks_david_schema'}

Para eliminar uno o todos los widgets:

In [0]:

#dbutils.widgets.remove('schema_name')
#dbutils.widgets.removeAll()

## Catalog

En Databricks, un Catalog es la capa más alta dentro del sistema de namespaces para organizar datos. Introducido con Unity Catalog, permite gestionar catálogos, esquemas y tablas de manera centralizada y segura, especialmente útil en entornos multiusuario y con integración de seguridad.

La jerarquía de objetos es:

Catalog > Schema (Database) > Table / View / Function

Cada catálogo puede contener múltiples esquemas, y cada esquema puede tener múltiples tablas o vistas.
Puedes usar SQL para crear un catálogo nuevo de esta forma:

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS sesion2;
-- CREATE CATALOG IF NOT EXISTS customer_cat COMMENT 'This is customer catalog';
-- CREATE CATALOG customer_cat MANAGED LOCATION 's3://depts/finance';
-- CREATE FOREIGN CATALOG postgresql_catalog USING CONNECTION postgresql_connection OPTIONS (database = 'my_db');

## Schemas


Un **Schema** (también conocido como Database en otros entornos) es un contenedor lógico dentro de un catálogo que agrupa tablas, vistas, funciones y otros objetos relacionados.  

In [0]:
%sql
-- En el esquema por defecto
CREATE SCHEMA IF NOT EXISTS schema_in_default_catalog

Puedes hacer tu notebook más flexible usando widgets para capturar el nombre del schema como parámetro:

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ${schema_name}

También puedes capturar el valor del widget en Python y construir dinámicamente la consulta SQL, lo cual, es mucho más flexible:

In [0]:
# Obtener el valor del widget
custom_schema = dbutils.widgets.get("schema_name")

# Crear la sentencia SQL con el nombre del schema
query = f"""
    CREATE SCHEMA IF NOT EXISTS {custom_schema}
"""

# Ejecutar el SQL dinámico
spark.sql(query)

DataFrame[]

Esto es útil cuando:
* Quieres hacer validaciones antes de ejecutar.
* El nombre del esquema depende de lógica adicional.
* Estás creando múltiples objetos de forma programática.

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS sesion2.${schema_name} COMMENT 'This is customer catalog';
-- CREATE SCHEMA customer_sc MANAGED LOCATION 's3://depts/finance';

Ahora vamos a obtener la información del esquema:

In [0]:
%sql
DESCRIBE SCHEMA EXTENDED ${catalog_name}.${schema_name};

database_description_item,database_description_value
Catalog Name,sesion2
Namespace Name,databricks_david_schema
Comment,This is customer catalog
Location,
Owner,tortilla-huso-43@icloud.com
Properties,
Predictive Optimization,ENABLE (inherited from METASTORE metastore_aws_eu_west_1)


## Volumes

Un **Volume** es un espacio de almacenamiento en un esquema (schema) de Unity Catalog que se utiliza para guardar archivos como CSV, JSON, Parquet, imágenes, etc.  
Es una forma estructurada y segura de trabajar con archivos dentro del entorno de Databricks, usando controles de acceso unificados (ACLs).

---

### 🔹 ¿Qué es un Volume?

- Son directorios montados dentro de Unity Catalog.
- Pueden ser **internos** (gestionados por Databricks) o **externos** (en un bucket S3 o ADLS).
- Útiles para manejar datos no tabulares o datos crudos que aún no han sido ingeridos en tablas.

In [0]:
%sql
-- CREATE VOLUME my_another_volume
CREATE VOLUME IF NOT EXISTS ${catalog_name}.${schema_name}.landing;
-- Create an external volume on the specified location with comment
-- CREATE EXTERNAL VOLUME my_catalog.my_schema.my_external_volume
-- LOCATION 's3://my-bucket/my-location/my-path'
-- COMMENT 'This is my example external volume on S3'

Ahora vamos a descargar un fichero CSV dentro del volumen, para ello, vamos a hacer uso de CURL:

In [0]:
%sh
curl -L https://raw.githubusercontent.com/dvddepennde/crops_training_school/refs/heads/main/nutrients_csvfile.csv -o /Volumes/sesion2/databricks_david_schema/landing/nutrients_csvfile.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19760  100 19760    0     0  38819      0 --:--:-- --:--:-- --:--:-- 38821


In [0]:
%sql
SELECT *
FROM csv.`/Volumes/${catalog_name}/${schema_name}/landing/nutrients_csvfile.csv`

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9
Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
Cows' milk,1 qt.,976,660,32,40,36,0,48,Dairy products
Milk skim,1 qt.,984,360,36,t,t,0,52,Dairy products
Buttermilk,1 cup,246,127,9,5,4,0,13,Dairy products
"Evaporated, undiluted",1 cup,252,345,16,20,18,0,24,Dairy products
Fortified milk,6 cups,"1,419","1,373",89,42,23,1.4,119,Dairy products
Powdered milk,1 cup,103,515,27,28,24,0,39,Dairy products
"skim, instant",1 1/3 cups,85,290,30,t,t,0,42,Dairy products
"skim, non-instant",2/3 cup,85,290,30,t,t,1,42,Dairy products
Goats' milk,1 cup,244,165,8,10,8,0,11,Dairy products


## Ejercicios dbutils & catalog & schema & volumes

1. Crea parámetros que indiquen el nombre del catálogo, nombre de schema y nombre de volumen. Asignales un valor que gustes, es el que utilizaremos para posteriores ejercicios.

2. Obten el valor de cada uno de los widgets y muestralos en una sola linea con el formato "Nombre de catálogo: {}, Nombre del esquema: {}, Nombre del volumen: {}"

3. Crea el catálogo, dentro de este, el esquema, y por último, dentro del esquema, el volumen. Haz las sentencias tanto en celdas SQL como en Python.

*IMPORTANTE*: Debes utilizar variables y no poner directamente el valor del parámetro en la sentencia

4. Descarga el archivo de la URL proporcionada dentro del volumen creado previamente. Una vez creado, deberás verificar que existe.

URL: https://raw.githubusercontent.com/dvddepennde/luxury-watches-analysis/refs/heads/main/data/Luxury%20watch.csv

Puede que la celda no sea necesario que sea en python...

5. Sube un CSV al volumen, de forma programatica o manual, para posterior análisis. Podéis buscar en https://www.kaggle.com/datasets . Una vez lo subas, consulta su contenido con SparkSQL.

6. Crea un notebook y llámalo sesion2_notebook_called, el cual, debe recibir por parámetro:
* db_name: Simulamos nombre de base de datos, si no sabéis que poner, poned "public".
* table_name: Simulamos nombre de tabla, si no sabéis que poner, poned vuestro nombre.
* num: Generar un número entero de forma aleatoria utilizando random.

El cual obtenga los valores, valide que "num" es un número y devolver:
* En caso de que la validación de num no sea correcta, un mensaje como el siguiente: "{'status': 'FAILED', 'custom_message': '<REPLACE_WITH_CUSTOM_MSJ>'}". El notebook no deberá continuar.
* En caso de que la validación sea correcta, un mensaje "Simulando operaciones..." y que devuelva: "{'status': 'OK', custom_message: '<REPLACE_WITH_CUSTOM_MSJ>', ... }" donde ... corresponde con devolver los mismos parámetros que recibió.

tip: Para validar si es o no entero, podéis usar este fragmento de código, o cualquiera que os parezca mejor:

```python3
try:
    # Intentamos convertirlo a entero
    num = int(num_str)
    # Código en caso de que SÍ sea entero

except ValueError:
    # Código si no es numérico (entero)
```


Buena suerte!


# Spark

Cuando PySpark lee un archivo (por ejemplo, un archivo CSV), puede intentar adivinar los tipos de datos de cada columna, basándose en los primeros registros del archivo. Este proceso se llama inferencia del esquema.

In [0]:
path_luxury_watches = f"/Volumes/{catalog}/{schema}/{volume}/luxury_watches.csv"
df = spark.read.csv(
    path=path_luxury_watches,
    header=True,     # Utiliza la primera fila como nombres de columnas
    inferSchema=True ## INFERENCIA DE ESQUEMA HABILITADA
)
display(df)

Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price (USD)
Rolex,Submariner,Stainless Steel,Stainless Steel,Automatic,300 meters,40.0,13.0,20.0,Black,Sapphire,Date,48 hours,"9,500"
Omega,Seamaster,Titanium,Rubber,Automatic,600 meters,43.5,14.47,21.0,Blue,Sapphire,Date,60 hours,"5,800"
Tag Heuer,Carrera,Stainless Steel,Leather,Automatic,100 meters,41.0,13.0,20.0,White,Sapphire,Chronograph,42 hours,"4,200"
Breitling,Navitimer,Stainless Steel,Stainless Steel,Automatic,30 meters,43.0,14.25,22.0,Black,Sapphire,Chronograph,70 hours,"7,900"
Cartier,Tank Solo,Stainless Steel,Leather,Quartz,30 meters,31.0,6.05,20.0,Silver,Sapphire,None,N/A,"2,800"
Jaeger-LeCoultre,Reverso,Stainless Steel,Leather,Manual,30 meters,42.9,9.2,20.0,Black,Sapphire,None,45 hours,"5,500"
Seiko,Prospex,Stainless Steel,Rubber,Automatic,200 meters,44.3,12.9,20.0,Black,Sapphire,Date,50 hours,"1,400"
Citizen,Promaster,Stainless Steel,Stainless Steel,Eco-Drive,200 meters,42.0,13.0,22.0,Black,Mineral,Chronograph,270 days,"1,200"
Tissot,Le Locle,Stainless Steel,Leather,Automatic,30 meters,39.3,9.75,19.0,White,Sapphire,Date,38 hours,650
Hamilton,Khaki Field,Stainless Steel,Leather,Automatic,100 meters,38.0,9.8,20.0,Black,Sapphire,None,80 hours,495


Ahora, en este caso, vamos a declarar nosotros el esquema que va a tener el dataframe:

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# Definir el esquema manualmente
schema = StructType([
    # Nombre, Tipo de dato, Requerido
    StructField("Brand", StringType(), True),
    StructField("Model", StringType(), True),
    StructField("Case Material", StringType(), True),
    StructField("Strap Material", StringType(), True),
    StructField("Movement Type", StringType(), True),
    StructField("Water Resistance", StringType(), True),
    StructField("Case Diameter (mm)", DoubleType(), True),
    StructField("Case Thickness (mm)", DoubleType(), True),
    StructField("Band Width (mm)", DoubleType(), True),
    StructField("Dial Color", StringType(), True),
    StructField("Crystal Material", StringType(), True),
    StructField("Complications", StringType(), True),
    StructField("Power Reserve", StringType(), True),
    StructField("Price (USD)", StringType(), True)
])

# Cargar el archivo CSV usando el esquema definido manualmente
df = spark.read.csv(path=path_luxury_watches, header=True, schema=schema)

# Mostrar los datos
display(df)


Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price (USD)
Rolex,Submariner,Stainless Steel,Stainless Steel,Automatic,300 meters,40.0,13.0,20.0,Black,Sapphire,Date,48 hours,"9,500"
Omega,Seamaster,Titanium,Rubber,Automatic,600 meters,43.5,14.47,21.0,Blue,Sapphire,Date,60 hours,"5,800"
Tag Heuer,Carrera,Stainless Steel,Leather,Automatic,100 meters,41.0,13.0,20.0,White,Sapphire,Chronograph,42 hours,"4,200"
Breitling,Navitimer,Stainless Steel,Stainless Steel,Automatic,30 meters,43.0,14.25,22.0,Black,Sapphire,Chronograph,70 hours,"7,900"
Cartier,Tank Solo,Stainless Steel,Leather,Quartz,30 meters,31.0,6.05,20.0,Silver,Sapphire,None,N/A,"2,800"
Jaeger-LeCoultre,Reverso,Stainless Steel,Leather,Manual,30 meters,42.9,9.2,20.0,Black,Sapphire,None,45 hours,"5,500"
Seiko,Prospex,Stainless Steel,Rubber,Automatic,200 meters,44.3,12.9,20.0,Black,Sapphire,Date,50 hours,"1,400"
Citizen,Promaster,Stainless Steel,Stainless Steel,Eco-Drive,200 meters,42.0,13.0,22.0,Black,Mineral,Chronograph,270 days,"1,200"
Tissot,Le Locle,Stainless Steel,Leather,Automatic,30 meters,39.3,9.75,19.0,White,Sapphire,Date,38 hours,650
Hamilton,Khaki Field,Stainless Steel,Leather,Automatic,100 meters,38.0,9.8,20.0,Black,Sapphire,None,80 hours,495


En cuanto a las diferencias, es importante tener en cuenta las siguientes características:


| Característica               | Inferir Esquema (automático)         | Definir Esquema (manual)         |
|------------------------------|--------------------------------------|----------------------------------|
| **Facilidad de uso**          | Muy fácil, no requiere intervención. | Requiere más trabajo inicial.   |
| **Precisión**                 | Puede ser impreciso con datos inconsistentes. | Alta precisión (100% control).  |
| **Rendimiento**               | Más lento en archivos grandes.       | Más rápido, ya que no necesita inferir. |
| **Flexibilidad**              | Ideal para datos que varían frecuentemente. | Menos flexible, requiere actualización manual. |
| **Robustez**                  | Puede fallar en ciertos casos (por ejemplo, valores no numéricos). | Es más robusto y seguro.       |


Pero...
- ¿Qué pasa si el esquema no coincide?: Si el esquema no coincide con el origen, Spark puede aceptar los datos dejando valores nulos (permissive), descartar las filas corruptas (dropMalformed) o fallar inmediatamente (failFast) según spark.read.option("mode", …).

- ¿Que ocurre si una de las columnas declaradas no existe?: Si el esquema declara una columna que no existe en los datos, Spark la crea y la rellena siempre con null (no lanza excepción)
- ¿Que ocurre si en el origen hay más columnas que las definidas en el esquema?: Si los datos tienen columnas extras que no están en el esquema, Spark las ignora y solo carga las definidas (dependiendo de enforceSchema).
- ¿Cuando se debería usar un esquema fijo y cuando inferirlo?: Se recomienda usar esquema fijo cuando se requiere alto rendimiento, consistencia y control estricto; usar inferSchema en etapas exploratorias o con datos muy variables para ganar flexibilidad.

Mostramos el esquema de las columnas:

In [0]:
df.printSchema()

root
 |-- Brand: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Case Material: string (nullable = true)
 |-- Strap Material: string (nullable = true)
 |-- Movement Type: string (nullable = true)
 |-- Water Resistance: string (nullable = true)
 |-- Case Diameter (mm): double (nullable = true)
 |-- Case Thickness (mm): double (nullable = true)
 |-- Band Width (mm): double (nullable = true)
 |-- Dial Color: string (nullable = true)
 |-- Crystal Material: string (nullable = true)
 |-- Complications: string (nullable = true)
 |-- Power Reserve: string (nullable = true)
 |-- Price (USD): string (nullable = true)



Mostrar únicamente las N primeras filas:

In [0]:
display(df.show(5))

+---------+----------+---------------+---------------+-------------+----------------+------------------+-------------------+---------------+----------+----------------+-------------+-------------+-----------+
|    Brand|     Model|  Case Material| Strap Material|Movement Type|Water Resistance|Case Diameter (mm)|Case Thickness (mm)|Band Width (mm)|Dial Color|Crystal Material|Complications|Power Reserve|Price (USD)|
+---------+----------+---------------+---------------+-------------+----------------+------------------+-------------------+---------------+----------+----------------+-------------+-------------+-----------+
|    Rolex|Submariner|Stainless Steel|Stainless Steel|    Automatic|      300 meters|              40.0|               13.0|           20.0|     Black|        Sapphire|         Date|     48 hours|      9,500|
|    Omega| Seamaster|       Titanium|         Rubber|    Automatic|      600 meters|              43.5|              14.47|           21.0|      Blue|        Sapph

Obtener las dimensiones del DataFrame:

In [0]:
rows = df.count()
columns = len(df.columns)
print(f"Número de filas: {rows}, columnas: {columns}")


Número de filas: 507, columnas: 14


Obtener lista de columnas del DataFrame

In [0]:
print(df.columns)


['Brand', 'Model', 'Case Material', 'Strap Material', 'Movement Type', 'Water Resistance', 'Case Diameter (mm)', 'Case Thickness (mm)', 'Band Width (mm)', 'Dial Color', 'Crystal Material', 'Complications', 'Power Reserve', 'Price (USD)']


Convertir el dataframe a dataframe de Pandas:

In [0]:
pandas_df = df.toPandas()
pandas_df.sample(4)

,Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price (USD)
18,Breitling,Superocean,Stainless Steel,Rubber,Automatic,500 meters,42.0,13.30,20.0,Blue,Sapphire,Date,38 hours,"3,200"
466,Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.0,8.30,22.0,Blue,Sapphire,Date,45 hours,"36,000"
461,Vacheron Constantin,Patrimony,White Gold,Leather,Manual,30 meters,40.0,6.79,20.0,Silver,Sapphire,None,65 hours,"28,000"
403,Hublot,Big Bang,Titanium,Rubber,Automatic,100 meters,44.0,14.50,25.0,Black,Sapphire,Chronograph,72 hours,"14,000"


Ordenar por alguna columna, de forma ascendente/descendente:

In [0]:
display(df.orderBy("Case Diameter (mm)", ascending=True).show(5))
## Lo cual es idéntico a
# display(df.sort("Case Diameter (mm)", ascending=True).show(5)


+-------+-------------+---------------+---------------+-------------+----------------+------------------+-------------------+---------------+----------+----------------+-------------+-------------+-----------+
|  Brand|        Model|  Case Material| Strap Material|Movement Type|Water Resistance|Case Diameter (mm)|Case Thickness (mm)|Band Width (mm)|Dial Color|Crystal Material|Complications|Power Reserve|Price (USD)|
+-------+-------------+---------------+---------------+-------------+----------------+------------------+-------------------+---------------+----------+----------------+-------------+-------------+-----------+
|Cartier|Santos-Dumont|18K Yellow Gold|        Leather|       Manual|       30 meters|              27.5|                6.5|           18.0|    Silver|        Sapphire|         None|          N/A|     14,200|
|Chopard|  Happy Sport|Stainless Steel|        Leather|       Quartz|       30 meters|              30.0|               10.5|           17.0|     White|        

Ordenar por varias columnas:

In [0]:
display(df.sort(['Band Width (mm)',"Case Diameter (mm)"], ascending=[False, False]).show(5))
## Lo cual es idéntico a
# display(df.orderBy(['Band Width (mm)',"Case Diameter (mm)"], ascending=[False, False]).show(5))

+----------------+--------------+---------------+---------------+-------------+----------------+------------------+-------------------+---------------+----------+----------------+-------------+-------------+-----------+
|           Brand|         Model|  Case Material| Strap Material|Movement Type|Water Resistance|Case Diameter (mm)|Case Thickness (mm)|Band Width (mm)|Dial Color|Crystal Material|Complications|Power Reserve|Price (USD)|
+----------------+--------------+---------------+---------------+-------------+----------------+------------------+-------------------+---------------+----------+----------------+-------------+-------------+-----------+
|         Bulgari|Octo Finissimo|       Titanium|        Leather|    Automatic|      100 meters|              40.0|               5.15|           28.0|     Black|        Sapphire|         None|     60 hours|     12,800|
|         Bulgari|Octo Finissimo|       Titanium|       Titanium|    Automatic|       30 meters|              40.0|     

Renombramos columna:

In [0]:
df = df.withColumnRenamed("Price (USD)", "Price")
# from pyspark.sql.functions import col
# df = df.select(col("Name").alias("name"), col("askdaosdka").alias("age"))
display(df)


Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price
Rolex,Submariner,Stainless Steel,Stainless Steel,Automatic,300 meters,40.0,13.0,20.0,Black,Sapphire,Date,48 hours,"9,500"
Omega,Seamaster,Titanium,Rubber,Automatic,600 meters,43.5,14.47,21.0,Blue,Sapphire,Date,60 hours,"5,800"
Tag Heuer,Carrera,Stainless Steel,Leather,Automatic,100 meters,41.0,13.0,20.0,White,Sapphire,Chronograph,42 hours,"4,200"
Breitling,Navitimer,Stainless Steel,Stainless Steel,Automatic,30 meters,43.0,14.25,22.0,Black,Sapphire,Chronograph,70 hours,"7,900"
Cartier,Tank Solo,Stainless Steel,Leather,Quartz,30 meters,31.0,6.05,20.0,Silver,Sapphire,None,N/A,"2,800"
Jaeger-LeCoultre,Reverso,Stainless Steel,Leather,Manual,30 meters,42.9,9.2,20.0,Black,Sapphire,None,45 hours,"5,500"
Seiko,Prospex,Stainless Steel,Rubber,Automatic,200 meters,44.3,12.9,20.0,Black,Sapphire,Date,50 hours,"1,400"
Citizen,Promaster,Stainless Steel,Stainless Steel,Eco-Drive,200 meters,42.0,13.0,22.0,Black,Mineral,Chronograph,270 days,"1,200"
Tissot,Le Locle,Stainless Steel,Leather,Automatic,30 meters,39.3,9.75,19.0,White,Sapphire,Date,38 hours,650
Hamilton,Khaki Field,Stainless Steel,Leather,Automatic,100 meters,38.0,9.8,20.0,Black,Sapphire,None,80 hours,495


Añadir columna como un literal:

In [0]:
from pyspark.sql.functions import lit
df = df.withColumn("Currency", lit("USD"))
df = df.withColumn("File", lit(f"/Volumes/{catalog}/{schema}/{volume}/luxury_watches.csv"))
display(df)

Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price,Currency,File
Rolex,Submariner,Stainless Steel,Stainless Steel,Automatic,300 meters,40.0,13.0,20.0,Black,Sapphire,Date,48 hours,"9,500",USD,"/Volumes/sesion2/StructType([StructField('Brand', StringType(), True), StructField('Model', StringType(), True), StructField('Case Material', StringType(), True), StructField('Strap Material', StringType(), True), StructField('Movement Type', StringType(), True), StructField('Water Resistance', StringType(), True), StructField('Case Diameter (mm)', DoubleType(), True), StructField('Case Thickness (mm)', DoubleType(), True), StructField('Band Width (mm)', DoubleType(), True), StructField('Dial Color', StringType(), True), StructField('Crystal Material', StringType(), True), StructField('Complications', StringType(), True), StructField('Power Reserve', StringType(), True), StructField('Price (USD)', StringType(), True)])/demo_volume/luxury_watches.csv"
Omega,Seamaster,Titanium,Rubber,Automatic,600 meters,43.5,14.47,21.0,Blue,Sapphire,Date,60 hours,"5,800",USD,"/Volumes/sesion2/StructType([StructField('Brand', StringType(), True), StructField('Model', StringType(), True), StructField('Case Material', StringType(), True), StructField('Strap Material', StringType(), True), StructField('Movement Type', StringType(), True), StructField('Water Resistance', StringType(), True), StructField('Case Diameter (mm)', DoubleType(), True), StructField('Case Thickness (mm)', DoubleType(), True), StructField('Band Width (mm)', DoubleType(), True), StructField('Dial Color', StringType(), True), StructField('Crystal Material', StringType(), True), StructField('Complications', StringType(), True), StructField('Power Reserve', StringType(), True), StructField('Price (USD)', StringType(), True)])/demo_volume/luxury_watches.csv"
Tag Heuer,Carrera,Stainless Steel,Leather,Automatic,100 meters,41.0,13.0,20.0,White,Sapphire,Chronograph,42 hours,"4,200",USD,"/Volumes/sesion2/StructType([StructField('Brand', StringType(), True), StructField('Model', StringType(), True), StructField('Case Material', StringType(), True), StructField('Strap Material', StringType(), True), StructField('Movement Type', StringType(), True), StructField('Water Resistance', StringType(), True), StructField('Case Diameter (mm)', DoubleType(), True), StructField('Case Thickness (mm)', DoubleType(), True), StructField('Band Width (mm)', DoubleType(), True), StructField('Dial Color', StringType(), True), StructField('Crystal Material', StringType(), True), StructField('Complications', StringType(), True), StructField('Power Reserve', StringType(), True), StructField('Price (USD)', StringType(), True)])/demo_volume/luxury_watches.csv"
Breitling,Navitimer,Stainless Steel,Stainless Steel,Automatic,30 meters,43.0,14.25,22.0,Black,Sapphire,Chronograph,70 hours,"7,900",USD,"/Volumes/sesion2/StructType([StructField('Brand', StringType(), True), StructField('Model', StringType(), True), StructField('Case Material', StringType(), True), StructField('Strap Material', StringType(), True), StructField('Movement Type', StringType(), True), StructField('Water Resistance', StringType(), True), StructField('Case Diameter (mm)', DoubleType(), True), StructField('Case Thickness (mm)', DoubleType(), True), StructField('Band Width (mm)', DoubleType(), True), StructField('Dial Color', StringType(), True), StructField('Crystal Material', StringType(), True), StructField('Complications', StringType(), True), StructField('Power Reserve', StringType(), True), StructField('Price (USD)', StringType(), True)])/demo_volume/luxury_watches.csv"
Cartier,Tank Solo,Stainless Steel,Leather,Quartz,30 meters,31.0,6.05,20.0,Silver,Sapphire,None,N/A,"2,800",USD,"/Volumes/sesion2/StructType([StructField('Brand', StringType(), True), StructField('Model', StringType(), True), StructField('Case Material'

Eliminamos columna:

In [0]:
df = df.drop("File")
display(df.limit(3))

Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price,Currency
Rolex,Submariner,Stainless Steel,Stainless Steel,Automatic,300 meters,40.0,13.0,20.0,Black,Sapphire,Date,48 hours,"9,500",USD
Omega,Seamaster,Titanium,Rubber,Automatic,600 meters,43.5,14.47,21.0,Blue,Sapphire,Date,60 hours,"5,800",USD
Tag Heuer,Carrera,Stainless Steel,Leather,Automatic,100 meters,41.0,13.0,20.0,White,Sapphire,Chronograph,42 hours,"4,200",USD


Obtenemos únicamente las columnas que nos resultan interesantes

In [0]:
display(df.select("Brand", "Model", "Price"))

Brand,Model,Price
Rolex,Submariner,"9,500"
Omega,Seamaster,"5,800"
Tag Heuer,Carrera,"4,200"
Breitling,Navitimer,"7,900"
Cartier,Tank Solo,"2,800"
Jaeger-LeCoultre,Reverso,"5,500"
Seiko,Prospex,"1,400"
Citizen,Promaster,"1,200"
Tissot,Le Locle,650
Hamilton,Khaki Field,495


Databricks visualization. Run in Databricks to view.

Mostramos los valores únicos de una columna

In [0]:
df.select("Brand").distinct().show()

+-------------------+
|              Brand|
+-------------------+
|          Blancpain|
|             Piaget|
|   A. Lange & Sohne|
|               Oris|
|             Zenith|
|      Ulysse Nardin|
|Frederique Constant|
|                IWC|
|             Hublot|
|            Bulgari|
|          Breitling|
|          TAG Heuer|
|              Rolex|
|    Audemars Piguet|
|            Chopard|
|        Bell & Ross|
|              Tudor|
|               Rado|
|   Jaeger-LeCoultre|
|   Girard-Perregaux|
+-------------------+
only showing top 20 rows


Unimos la columna Brand con el model:


In [0]:
from pyspark.sql.functions import concat_ws, col
df = df.withColumn(
    "Brand_Model",
    concat_ws(" ", col("Brand"), col("Model"))  # Concatenar 'Brand' y 'Model' con un espacio en el medio
)
display(df)

Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price,Currency,Brand_Model
Rolex,Submariner,Stainless Steel,Stainless Steel,Automatic,300 meters,40.0,13.0,20.0,Black,Sapphire,Date,48 hours,"9,500",USD,Rolex Submariner
Omega,Seamaster,Titanium,Rubber,Automatic,600 meters,43.5,14.47,21.0,Blue,Sapphire,Date,60 hours,"5,800",USD,Omega Seamaster
Tag Heuer,Carrera,Stainless Steel,Leather,Automatic,100 meters,41.0,13.0,20.0,White,Sapphire,Chronograph,42 hours,"4,200",USD,Tag Heuer Carrera
Breitling,Navitimer,Stainless Steel,Stainless Steel,Automatic,30 meters,43.0,14.25,22.0,Black,Sapphire,Chronograph,70 hours,"7,900",USD,Breitling Navitimer
Cartier,Tank Solo,Stainless Steel,Leather,Quartz,30 meters,31.0,6.05,20.0,Silver,Sapphire,None,N/A,"2,800",USD,Cartier Tank Solo
Jaeger-LeCoultre,Reverso,Stainless Steel,Leather,Manual,30 meters,42.9,9.2,20.0,Black,Sapphire,None,45 hours,"5,500",USD,Jaeger-LeCoultre Reverso
Seiko,Prospex,Stainless Steel,Rubber,Automatic,200 meters,44.3,12.9,20.0,Black,Sapphire,Date,50 hours,"1,400",USD,Seiko Prospex
Citizen,Promaster,Stainless Steel,Stainless Steel,Eco-Drive,200 meters,42.0,13.0,22.0,Black,Mineral,Chronograph,270 days,"1,200",USD,Citizen Promaster
Tissot,Le Locle,Stainless Steel,Leather,Automatic,30 meters,39.3,9.75,19.0,White,Sapphire,Date,38 hours,650,USD,Tissot Le Locle
Hamilton,Khaki Field,Stainless Steel,Leather,Automatic,100 meters,38.0,9.8,20.0,Black,Sapphire,None,80 hours,495,USD,Hamilton Khaki Field


Convertimos la columna Price a numérico

In [0]:
from pyspark.sql.functions import col, regexp_replace
df_cleaned = df.withColumn(
    "Price",
    regexp_replace(col("Price"), r"[^\d.]", "")  # Eliminar todo lo que no sea dígito o punto
).withColumn(
    "Price",
    col("Price").cast("double")  # Convertir a tipo numérico (Double)
)
df_cleaned.printSchema()

root
 |-- Brand: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Case Material: string (nullable = true)
 |-- Strap Material: string (nullable = true)
 |-- Movement Type: string (nullable = true)
 |-- Water Resistance: string (nullable = true)
 |-- Case Diameter (mm): double (nullable = true)
 |-- Case Thickness (mm): double (nullable = true)
 |-- Band Width (mm): double (nullable = true)
 |-- Dial Color: string (nullable = true)
 |-- Crystal Material: string (nullable = true)
 |-- Complications: string (nullable = true)
 |-- Power Reserve: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Currency: string (nullable = false)
 |-- Brand_Model: string (nullable = false)



Vamos ahora a ordenar por precio del reloj:

In [0]:
display(df_cleaned.orderBy("Price", ascending=False).limit(10))


Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price,Currency,Brand_Model
Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.5,8.3,22.0,Blue,Sapphire,"Date, Moon Phase",45 hours,70000.0,USD,Patek Philippe Nautilus
Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.5,8.3,20.0,Blue,Sapphire,"Date, Chronograph",55 hours,67000.0,USD,Patek Philippe Nautilus
Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.8,8.3,21.0,Blue,Sapphire,Date,45 hours,62500.0,USD,Patek Philippe Nautilus
Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.8,8.3,20.0,Blue,Sapphire,Date,45 hours,57000.0,USD,Patek Philippe Nautilus
Audemars Piguet,Royal Oak,Rose Gold,Rose Gold,Automatic,50 meters,41.0,9.8,20.0,Blue,Sapphire,Date,60 hours,55000.0,USD,Audemars Piguet Royal Oak
Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.8,8.3,20.0,Blue,Sapphire,Date,45 hours,52000.0,USD,Patek Philippe Nautilus
Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.5,8.3,22.0,Blue,Sapphire,Date,45 hours,51000.0,USD,Patek Philippe Nautilus
Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.8,8.3,21.0,Blue,Sapphire,Date,55 hours,51000.0,USD,Patek Philippe Nautilus
Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.5,8.3,20.0,Blue,Sapphire,Date,45 hours,49800.0,USD,Patek Philippe Nautilus
Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.8,8.3,20.0,Blue,Sapphire,Date,45 hours,47000.0,USD,Patek Philippe Nautilus


### Filtros


Filtramos por un valor exacto de tipo String

In [0]:
# Filtrar los registros donde la columna 'Movement Type' tiene el valor 'Automatic'
df_filtered = df_cleaned.filter(df_cleaned['Movement Type'] == 'Automatic')

# Mostrar las primeras 5 filas
display(df_filtered.limit(5))

Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price,Currency,Brand_Model
Rolex,Submariner,Stainless Steel,Stainless Steel,Automatic,300 meters,40.0,13.0,20.0,Black,Sapphire,Date,48 hours,9500.0,USD,Rolex Submariner
Omega,Seamaster,Titanium,Rubber,Automatic,600 meters,43.5,14.47,21.0,Blue,Sapphire,Date,60 hours,5800.0,USD,Omega Seamaster
Tag Heuer,Carrera,Stainless Steel,Leather,Automatic,100 meters,41.0,13.0,20.0,White,Sapphire,Chronograph,42 hours,4200.0,USD,Tag Heuer Carrera
Breitling,Navitimer,Stainless Steel,Stainless Steel,Automatic,30 meters,43.0,14.25,22.0,Black,Sapphire,Chronograph,70 hours,7900.0,USD,Breitling Navitimer
Seiko,Prospex,Stainless Steel,Rubber,Automatic,200 meters,44.3,12.9,20.0,Black,Sapphire,Date,50 hours,1400.0,USD,Seiko Prospex


Filtramos con condición de cadena:

In [0]:
display(df_cleaned.filter(
  (df_cleaned['Case Material'].contains('Steel')) & 
  ~(df_cleaned['Strap Material'].contains('eather')) &
  (df_cleaned['Water Resistance'].contains('100'))
))

Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price,Currency,Brand_Model
Oris,Big Crown ProPilot,Stainless Steel,Textile,Automatic,100 meters,41.0,12.0,20.0,Black,Sapphire,"Date, GMT",38 hours,1800.0,USD,Oris Big Crown ProPilot
Bell & Ross,Aviation,Stainless Steel,Rubber,Automatic,100 meters,42.0,11.5,24.0,Black,Sapphire,Chronograph,42 hours,4500.0,USD,Bell & Ross Aviation
Rolex,GMT-Master II,Stainless Steel,Stainless Steel,Automatic,100 meters,40.0,12.5,20.0,Black,Sapphire,GMT,48 hours,14000.0,USD,Rolex GMT-Master II
Rolex,Datejust,Stainless Steel,Jubilee,Automatic,100 meters,36.0,12.0,20.0,Silver,Sapphire,Date,70 hours,9000.0,USD,Rolex Datejust
Zenith,El Primero,Stainless Steel,Alligator,Automatic,100 meters,42.0,12.75,20.0,Silver,Sapphire,Chronograph,50 hours,6500.0,USD,Zenith El Primero
Piaget,Polo S,Stainless Steel,Stainless Steel,Automatic,100 meters,42.0,9.4,24.0,Blue,Sapphire,Date,50 hours,10000.0,USD,Piaget Polo S
Ulysse Nardin,Marine,Stainless Steel,Rubber,Automatic,100 meters,44.0,12.5,22.0,Blue,Sapphire,Date,60 hours,9500.0,USD,Ulysse Nardin Marine
Girard-Perregaux,Laureato,Stainless Steel,Stainless Steel,Automatic,100 meters,42.0,10.88,20.0,Blue,Sapphire,None,54 hours,7800.0,USD,Girard-Perregaux Laureato
Girard-Perregaux,Laureato,Stainless Steel,Stainless Steel,Automatic,100 meters,38.0,10.88,25.0,Blue,Sapphire,None,54 hours,6700.0,USD,Girard-Perregaux Laureato
Rolex,Datejust,Stainless Steel,Stainless Steel,Automatic,100 meters,41.0,11.2,20.0,Silver,Sapphire,Date,70 hours,8900.0,USD,Rolex Datejust


Filtramos con operador mayor que:

In [0]:
# Filtrar los registros donde la columna 'Price' es mayor que 98
df_filtered = df_cleaned.filter(df_cleaned['Price'] > 65000)
display(df_filtered)

Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price,Currency,Brand_Model
Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.5,8.3,22.0,Blue,Sapphire,"Date, Moon Phase",45 hours,70000.0,USD,Patek Philippe Nautilus
Patek Philippe,Nautilus,Stainless Steel,Stainless Steel,Automatic,120 meters,40.5,8.3,20.0,Blue,Sapphire,"Date, Chronograph",55 hours,67000.0,USD,Patek Philippe Nautilus


Condiciones múltiples:

In [0]:
df_filtered = df_cleaned.filter(
  (df_cleaned['Brand'] == 'Rolex') & 
  (df_cleaned['Movement Type'] == 'Automatic') & 
  (df_cleaned['Power Reserve'] == '55 hours') & 
  (df_cleaned['Price'] > 65000)
)
display(df_filtered)


Brand,Model,Case Material,Strap Material,Movement Type,Water Resistance,Case Diameter (mm),Case Thickness (mm),Band Width (mm),Dial Color,Crystal Material,Complications,Power Reserve,Price,Currency,Brand_Model


### Agrupaciones

Contar los relojes por marca

In [0]:
display(df_cleaned.groupBy('Brand').count().withColumnRenamed("count", "Nº de relojes").orderBy('Nº de relojes', ascending=False).show(5))

+---------------+-------------+
|          Brand|Nº de relojes|
+---------------+-------------+
|            IWC|           39|
|Audemars Piguet|           38|
| Patek Philippe|           33|
|         Zenith|           30|
|      Blancpain|           29|
+---------------+-------------+
only showing top 5 rows


Media de precio por marca y modelo

In [0]:
#display(df_cleaned.groupBy(['Brand', 'Model']).avg())
#display(df_cleaned.select(['Brand', 'Model' , 'Price']).groupBy(['Brand', 'Model']).avg())
display(
    df_cleaned.select([
        'Brand', 'Model' , 'Price']
    ).groupBy(
        ['Brand', 'Model']
    ).avg().withColumnRenamed(
        "avg(Price)", "AVG Price"
    ).sort(['AVG Price', 'Brand'], ascending=[False, True]).show(5)
)

+----------------+---------+------------------+
|           Brand|    Model|         AVG Price|
+----------------+---------+------------------+
|  Patek Philippe| Nautilus|42230.769230769234|
|           Rolex|  Daytona|           27500.0|
| Audemars Piguet|Royal Oak| 23734.21052631579|
|  Patek Philippe|Calatrava|23385.714285714286|
|A. Lange & Sohne|  Saxonia|           23000.0|
+----------------+---------+------------------+
only showing top 5 rows


### Tabla

In [0]:
catalog = dbutils.widgets.get('catalog_name')
schema = dbutils.widgets.get('schema_name')
table_name = "luxury_watches"

## Renombramos columnas, ya que no se aceptan acentos, ni espacios, establecemos snake_case

df_cleaned = df_cleaned.withColumnRenamed("Case Material", "case_material").withColumnRenamed("Strap Material", "strap_material").withColumnRenamed("Movement Type", "movement_type").withColumnRenamed("Water Resistance", "water_resistance").withColumnRenamed("Case Diameter (mm)", "case_diameter").withColumnRenamed("Case Thickness (mm)", "case_thickness").withColumnRenamed("Band Width (mm)", "band_width").withColumnRenamed("Dial Color", "dial_color").withColumnRenamed("Crystal Material", "crystal_material").withColumnRenamed("Complications", "complications").withColumnRenamed("Power Reserve", "power_reserve").withColumnRenamed("Price", "price").withColumnRenamed("Currency", "currency").withColumnRenamed("Brand_Model", "brand_model")

df_cleaned.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{table_name}")
print(f"Tabla creada: {catalog}.{schema}.{table_name}")

Tabla creada: sesion2.databricks_david_schema.luxury_watches


Consultar la tabla creada:


In [0]:
data = spark.sql(f"""SELECT * FROM {catalog}.{schema}.{table_name}""")
display(data)

Brand,Model,case_material,strap_material,movement_type,water_resistance,case_diameter,case_thickness,band_width,dial_color,crystal_material,complications,power_reserve,price,currency,brand_model
Rolex,Submariner,Stainless Steel,Stainless Steel,Automatic,300 meters,40.0,13.0,20.0,Black,Sapphire,Date,48 hours,9500.0,USD,Rolex Submariner
Omega,Seamaster,Titanium,Rubber,Automatic,600 meters,43.5,14.47,21.0,Blue,Sapphire,Date,60 hours,5800.0,USD,Omega Seamaster
Tag Heuer,Carrera,Stainless Steel,Leather,Automatic,100 meters,41.0,13.0,20.0,White,Sapphire,Chronograph,42 hours,4200.0,USD,Tag Heuer Carrera
Breitling,Navitimer,Stainless Steel,Stainless Steel,Automatic,30 meters,43.0,14.25,22.0,Black,Sapphire,Chronograph,70 hours,7900.0,USD,Breitling Navitimer
Cartier,Tank Solo,Stainless Steel,Leather,Quartz,30 meters,31.0,6.05,20.0,Silver,Sapphire,None,N/A,2800.0,USD,Cartier Tank Solo
Jaeger-LeCoultre,Reverso,Stainless Steel,Leather,Manual,30 meters,42.9,9.2,20.0,Black,Sapphire,None,45 hours,5500.0,USD,Jaeger-LeCoultre Reverso
Seiko,Prospex,Stainless Steel,Rubber,Automatic,200 meters,44.3,12.9,20.0,Black,Sapphire,Date,50 hours,1400.0,USD,Seiko Prospex
Citizen,Promaster,Stainless Steel,Stainless Steel,Eco-Drive,200 meters,42.0,13.0,22.0,Black,Mineral,Chronograph,270 days,1200.0,USD,Citizen Promaster
Tissot,Le Locle,Stainless Steel,Leather,Automatic,30 meters,39.3,9.75,19.0,White,Sapphire,Date,38 hours,650.0,USD,Tissot Le Locle
Hamilton,Khaki Field,Stainless Steel,Leather,Automatic,100 meters,38.0,9.8,20.0,Black,Sapphire,None,80 hours,495.0,USD,Hamilton Khaki Field


### Ejercicios Spark

1. Filtrar por la marca Rolex

2. Consulta la tabla samples.tpch.orders y realiza algún tipo de gráfico (O alguna similar de samples.tpch o del CSV extraído)


3. Utilizando el CSV propio que has subido al volumen, crea una tabla en el catálogo creado y consultala con SQL. Prueba a añadir columnas adicionales al CSV, como por ejemplo, refresh_date a la fecha actual, nombre del fichero...

# Control de versiones + Best practiques  + Modularization
En este punto, vamos a unir nuestro repositorio de Github: https://github.com/dvddepennde/databricks_notebook_bp al Workspace y trabajaremos con él, realizando una serie de operaciones y mostrando como debería ser estructurado.